# [ Introduction](Introduction.ipynb)

# [ Données météorologiques](Donnes_meteo.ipynb)

# [ Modèle Thermique et modèle mathematique](Modele_thermique.ipynb)

# [Implementation et resultats : simulation stationnaire, polystrène expansé ](Stationnaire_cas.ipynb)

# [ Implementation et resultats : simulation stationnaire, laine de mouton ](laine.ipynb)

# Estimation en degrés-heure du potentiel d'économie d'énergie obtenu par le chauffage intermittent

L'intermittence consiste à réduire la puissance de chauffage pendant la nuit. En première approximation, les économies obtenues peuvent être estimées en utilisant les degrés-heures ([Ghiaus and Allard, 2006](https://doi.org/10.1016/j.solener.2005.05.019)).

Bien que les utilisations du projet ne soient pas entièrement définies, nous envisageons l'hypothèse que les petits appartements seront utilisés comme logements,  dans le but d'estimer les économies d'énergie par la méthode en degrés par heure.

On peut modeleliser le bâtiment comme un volume separé du reste de l'environnement par un envelope. Les températures interieurs et exterieurs ainsi que les propietés de l'envelope sont homogènes. On écrit l'équilibre stationnaire comme : 
$$\dot{Q}_{HVAC} - U_{bldg}·S·(\theta_{in} - \theta_{out}) = 0$$

The energy consumption of the HVAC system needed to maintain the indoor temperature at the value $\theta_{in}$ over a period of time is the integral in time of the power load ([Ghiaus, 2006](https://doi.org/10.1016/j.enbuild.2005.08.014)):

$$Q_{HVAC} = \int_{0}^{t} |\dot{Q}_{HVAC}| dt = \int_{0}^{t} |U_{bldg}·S·(\theta_{in} - \theta_{out})| dt $$

where $|\dot{Q}_{HVAC}|$ is the absolute value of $\dot{Q}_{HVAC}$.

On considére la temperature constante pendant la journée $(T_{day})$ et une temperature réduite pendant la nuit $(T_{night})$.

## Implementation dans Python

In [1]:
import pandas as pd
from dm4bem import read_epw

# Inputs
# ======
filename = './weather_data/FRA_AR_Bourg-St-Maurice.074970_TMYx.2007-2021.epw'

θ = 20          # °C, indoor temperature all time
θday = θ        # °C, indoor temperature during day,, e.g.: 06:00 - 22:00
θnight = 14     # °C, indoor temperature during night 23:00 - 06:00

#tout l'année
period_start = '2000-01-01'
period_end = '2000-12-31'

daytime_start = '06:00:00+01:00'
daytime_end = '22:00:00+01:00'

In [2]:
# Computation
# ===========
# read Energy Plus Weather data (file .EPW)
[data, meta] = read_epw(filename, coerce_year=2000)

# select outdoor air temperature; call it θout
df = data[["temp_air"]]
del data
df = df.rename(columns={'temp_air': 'θout'})

# Select the data for a period of the year
df = df.loc[period_start:period_end]

### Différences de température pour une température intérieure constante

$$\Delta \theta_{fix,k} = \theta - \left \lceil  \theta_{out,k} \right \rceil$$

où :
- $\theta$ - température intérieure (constante tout le temps), °C;
- $\left \lceil \theta_{out,k} \right \rceil$ - température extérieure limitée par la température intérieure (c'est-à-dire non supérieure à la température intérieure), °C,

où :
$\left \lceil  \theta_{out,k} \right \rceil = 
\begin{cases}
\theta_{out,k} \text{, if } \theta_{out,k} < \theta
\\
\theta \text{ otherwise}
\end{cases}$


In [3]:
# Compute degree-hours for fixed set-point
# ----------------------------------------
df['Δθfix'] = θ - df['θout'].where(
    df['θout'] < θ,
    θ)

### Différences de température pour une température intérieure variable

#### Jour et nuit

L'heure du jour s'étend du "début du jour" à la "fin du jour" (indiquée en "HH:00"). La nuit correspond à la période où il n'y a pas de jour.

In [4]:
# Define start time for day and night
day_start = pd.to_datetime(daytime_start).time()
day_end = pd.to_datetime(daytime_end).time()

# Daytime should be between 00:00 and 24:00
# Daytime including midnight is not allowed, e.g., 22:00 till 06:00
day = (df.index.time >= day_start) & (df.index.time <= day_end)
night = ~day

#### Différences de température pour le jour

$$\Delta \theta_{day,k} = \theta_{day} - \left \lceil  \theta_{out,k} \right \rceil_{day}$$

where:
- $\theta_{day}$ - indoor temperature over the time $k$, °C;
- $\left \lceil  \theta_{out,k} \right \rceil_{day}$ - outdoor temperature bounded by indoor temperature during the daytime, °C,

where 
$\left \lceil  \theta_{out,k} \right \rceil_{day} =
\begin{cases}
\theta_{out,k} \text{, if } \theta_{out,k} < \theta_{day} \text{ and daytime}
\\
\theta_{day} \text{, otherwise}
\end{cases}$ 


In [5]:
# Degree-hours for daytime
df['Δθday'] = θday - df['θout'].where(
    (df['θout'] < θday) & day,
    θday)

#### Différences de température pour la nuit

$$\Delta \theta_{night,k} = \theta_{night} - \left \lceil  \theta_{out,k} \right \rceil_{night}$$

where:
- $\theta_{night}$ - indoor temperature over the time $k$, °C;
- $\left \lceil  \theta_{out,k} \right \rceil_{night}$ - outdoor temperature bounded by the indoor temperature during the nighttime, °C, 

where
$\left \lceil  \theta_{out,k} \right \rceil_{night} =
\begin{cases}
\theta_{out,k} \text{, if }  \theta_{out,k} < \theta_{night}  \text{ and nighttime}
\\
\theta_{night} \text{, otherwise}
\end{cases}$ 

In [6]:
# Degree-hours for nighttime
df['Δθnight'] = θnight - df['θout'].where(
    (df['θout'] < θnight) & night,
    θnight)

### Degrés-heures de chauffage (DHH)

Les degrés-heures de chauffage (_DHH_) sont la somme des différences de température :
- pour une température intérieure fixe

$$DHH_{fix} = \sum_{k=0}^{n} \Delta \theta_{fix,k} = \sum_{k=0}^{n} ( \theta - \lceil gauche \theta_{out,k} \rceil droit) $$$

- pour la température intérieure de jour

$$DHH_{day} = \sum_{k=0}^{n} \Delta \theta_{day,k} = \sum_{k=0}^{n} ( \theta_{day} - \left \lceil \theta_{out,k} \right \rceil_{day})$$

- pour la température intérieure nocturne

$$DHH_{night} = \sum_{k=0}^{n} \Delta \theta_{night,k} = \sum_{k=0}^{n} ( \theta_{night} - \left \lceil \theta_{out,k} \right \rceil_{night})$$

- pour l'intermittence

$$DHH_{interm} = DHH_{day} + DHH_{night}$$.

Le pourcentage d'économie d'énergie par l'utilisation d'un chauffage intermittent est :

$$s = \frac{DHH_{fix} - DHH_{interm}}{DHH_{fix}} 100 \%$$

In [7]:
# Sum of degree-hours for fixed indoor temperature
DHH_fix = df['Δθfix'].sum()

# Sum of degree-hours for intermittent heating
DHH_interm = df['Δθday'].sum() + df['Δθnight'].sum()

# Results
# =======
print(f"degree-hours fixed set-point: {DHH_fix:.1f} h·K")
print(f"degree-hours variable set-point: {DHH_interm:.1f} h·K")
print(f"Estimated savings: {(DHH_fix - DHH_interm) / DHH_fix * 100:.0f} %")

degree-hours fixed set-point: 126222.4 h·K
degree-hours variable set-point: 110904.6 h·K
Estimated savings: 12 %


## Conclusion sur les résultats

Des économies d'énergie de 12 % sont attendues, ce qui permet de conclure qu'il est plus efficace d'utiliser un chauffage intermittent.

# [ Modèle Mathematique, implementation et resultats : simulation stationnaire à répresetation état-spatiel ](state-space.ipynb)

# [ Simulation dynamique ](simulation_dynamique.ipynb)

# [Simulation dynamique : changer entre modèles](heat-cool.ipynb)

# [Simulation dynamique : Contrôleur avec feedback](feedback_control.ipynb)


---